

---
# IPL 2023 - PowerPlay Prediction Regression Project
The aim of this project is to predict the runs that will be scored in the powerplay of each innings, given the infomration of the venue, batting an bowling team for that innings, and the players which participated in the given overs.

The metric to gauge error is mean absolute error.

---


#1) Importing the Dependencies

In [ ]:
import pandas as pd
import numpy as np
import hashlib

# 2) Loading the data in the environment

In [ ]:
data_ball = pd.read_csv('/content/IPL_Ball_by_Ball_2008_2022.csv')
data_match = pd.read_csv('/content/IPL_Matches_Result_2008_2022.csv')

In [ ]:
data_ball

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70474,1136582,2,7,5,SS Iyer,R Ashwin,RR Pant,NaN,1,0,1,0,0,NaN,NaN,NaN,Delhi Daredevils
70475,1136582,2,7,6,RR Pant,R Ashwin,SS Iyer,NaN,1,0,1,0,0,NaN,NaN,NaN,Delhi Daredevils
70476,1136582,2,8,1,RR Pant,Mujeeb Ur Rahman,SS Iyer,NaN,0,0,0,0,1,RR Pant,bowled,NaN,Delhi Daredevils
70477,1136582,2,8,2,DT Christian,Mujeeb Ur Rahman,SS Iyer,NaN,0,0,0,0,0,NaN,NaN,NaN,Delhi Daredevils


In [ ]:
data_match

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,335986,Kolkata,2008-04-20,2007/08,4,Kolkata Knight Riders,Deccan Chargers,Eden Gardens,Deccan Chargers,bat,N,Kolkata Knight Riders,Wickets,5.0,NaN,DJ Hussey,"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC G...","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...",BF Bowden,K Hariharan
946,335985,Mumbai,2008-04-20,2007/08,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,N,Royal Challengers Bangalore,Wickets,5.0,NaN,MV Boucher,"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', '...","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', '...",SJ Davis,DJ Harper
947,335984,Delhi,2008-04-19,2007/08,3,Delhi Daredevils,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,N,Delhi Daredevils,Wickets,9.0,NaN,MF Maharoof,"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwa...","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif'...",Aleem Dar,GA Pratapkumar
948,335983,Chandigarh,2008-04-19,2007/08,2,Kings XI Punjab,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,N,Chennai Super Kings,Runs,33.0,NaN,MEK Hussey,"['K Goel', 'JR Hopes', 'KC Sangakkara', 'Yuvra...","['PA Patel', 'ML Hayden', 'MEK Hussey', 'MS Dh...",MR Benson,SL Shastri


#3) PreProcessing

##a) Removing Redundant columns in Ball by Ball Data
As we can see, there are many columns which are redundant for our purpose, hence we have to remove those from our data set i.e. data_ball

In [ ]:
def drop_redundant_fea_bowl(data):
  #Storing all the redundant features in a List
  L = ['extra_type', 'batsman_run', 'extras_run','non_boundary', 'isWicketDelivery', 'player_out', 'kind',
        'fielders_involved','ballnumber','non-striker']
  #Using drop feature to remove them from our data
  data = data.drop(L,axis = 1)
  return data

data_ball = drop_redundant_fea_bowl(data_ball)

In [ ]:
data_ball.head()

,ID,innings,overs,batter,bowler,total_run,BattingTeam
0,1312200,1,0,YBK Jaiswal,Mohammed Shami,0,Rajasthan Royals
1,1312200,1,0,YBK Jaiswal,Mohammed Shami,1,Rajasthan Royals
2,1312200,1,0,JC Buttler,Mohammed Shami,1,Rajasthan Royals
3,1312200,1,0,YBK Jaiswal,Mohammed Shami,0,Rajasthan Royals
4,1312200,1,0,YBK Jaiswal,Mohammed Shami,0,Rajasthan Royals


Now, since our aim is to predict power play runs, it is important that we remove entries from the overs not in power play i.e. over 6-20 (14 overs).

In [ ]:
def powerplay_cap(data):
  #Selecting all overs from 0-6 (since indexing begins at 0, we select <= 5)
  data = data[data_ball['overs']<= 5]
  return data

data_ball = powerplay_cap(data_ball)

In [ ]:
data_ball.head()

,ID,innings,overs,batter,bowler,total_run,BattingTeam
0,1312200,1,0,YBK Jaiswal,Mohammed Shami,0,Rajasthan Royals
1,1312200,1,0,YBK Jaiswal,Mohammed Shami,1,Rajasthan Royals
2,1312200,1,0,JC Buttler,Mohammed Shami,1,Rajasthan Royals
3,1312200,1,0,YBK Jaiswal,Mohammed Shami,0,Rajasthan Royals
4,1312200,1,0,YBK Jaiswal,Mohammed Shami,0,Rajasthan Royals


## b) Create a merged dataset, which converts our data in the format of input data set
Here, we need to group our data based on ID of each match and innings. However, since most of the variables are categorical variables and hence, have string values, we will have to use 'lambda' function to join these values together and remove duplications

In [ ]:
def merge_strings_data(data):
  #We use agg() function to aggregate the column values based on grouping by 'ID' and 'innings'
  #Using ',' is a way by which we are cumilating these values with the de limiter of a ,
  #Now we use join(set(_)) to join these values by removing duplicates in the way
  #Also, since 'total_run' is a numeric value, we can use 'sum' straight away to aggregate those values
  merged = data.groupby(['ID', 'innings']).agg({'batter': lambda g: ','.join(set(g)),'bowler': lambda g: ','.join(set(g)),'BattingTeam': lambda g: ','.join(set(g)),'total_run': 'sum'}).reset_index()
  return merged

In [ ]:
merged = merge_strings_data(data_ball)

In [ ]:
merged

,ID,innings,batter,bowler,BattingTeam,total_run
0,1136582,1,"KK Nair,KL Rahul,AJ Finch,MA Agarwal","TA Boult,LE Plunkett,Avesh Khan",Kings XI Punjab,51
1,1136582,2,"G Gambhir,RR Pant,GJ Maxwell,SS Iyer,PP Shaw","BB Sran,AJ Tye,AS Rajpoot",Delhi Daredevils,48
2,1136583,1,"Shakib Al Hasan,KS Williamson,WP Saha,S Dhawan...","JJ Bumrah,HH Pandya,MJ McClenaghan",Sunrisers Hyderabad,51
3,1136583,2,"E Lewis,RG Sharma,Ishan Kishan,SA Yadav,KH Pandya","Mohammad Nabi,Sandeep Sharma,Shakib Al Hasan",Mumbai Indians,22
4,1136584,1,"Q de Kock,AB de Villiers,V Kohli","Harbhajan Singh,SN Thakur,DL Chahar",Royal Challengers Bangalore,52
...,...,...,...,...,...,...
597,1312198,2,"Q de Kock,KL Rahul,M Vohra,DJ Hooda","Mohammed Siraj,Shahbaz Ahmed,JR Hazlewood",Lucknow Super Giants,62
598,1312199,1,"RM Patidar,V Kohli,F du Plessis","TA Boult,M Prasidh Krishna",Royal Challengers Bangalore,46
599,1312199,2,"SV Samson,YBK Jaiswal,JC Buttler","Mohammed Siraj,Shahbaz Ahmed,GJ Maxwell,JR Haz...",Rajasthan Royals,67
600,1312200,1,"SV Samson,YBK Jaiswal,JC Buttler","LH Ferguson,Yash Dayal,Mohammed Shami,Rashid Khan",Rajasthan Royals,44


##c) Creating Common Database
Now, we can merge both our databases i.e. data_ball and data_match to get all the information we have available and use it from one common dataframe

In [ ]:
def merge_databases(data1,data2):
  #We use merge function to merge both the dataframes on the common column i.e. 'ID'
  data = pd.merge(data1,data2, on = ['ID'])
  return data

data = merge_databases(merged,data_match)


Now, we can also see that there are some redundant features in our final dataframe also, which we do not need in our initial stage. Hence we can remove those

In [ ]:
def drop_redundant_fea_match(data):
  #We store those redundant columns in a List and drop those from our dataframe
  L = ['WinningTeam', 'WonBy', 'Margin', 'method',
       'Player_of_Match','TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'method',
       'Player_of_Match','Umpire1',
       'Umpire2','City','Date','Season','MatchNumber','Team1Players','Team2Players']
  data = data.drop(L,axis = 1)
  return data

data = drop_redundant_fea_match(data)

In [ ]:
#Checking the shape of the dataframe till now
data.shape


(602, 9)

##d) Creating a bowling team column
Now, we need a column which has the bowling team for each particular innings, for which we will use Team1 and Team2 columns and compare it to batting team 

In [ ]:
def bowling_team_creation(data):
  if data['BattingTeam'] == data["Team1"]:
      return data["Team2"]
  if data["BattingTeam"] == data["Team2"]:
      return data["Team1"]

def bowl_team(data):
  #We use apply function and the function created above to add bowling team in our data set
  data['BowlingTeam'] = data.apply(bowling_team_creation,axis = 1)
  return data

data = bowl_team(data)

In [ ]:
#Now, we can drop Team1 and Team2 from our data, as their work is now done
data = data.drop(['Team1','Team2'],axis = 1)

## e) Renaming & ReIndexing 

In [ ]:
def rename_reindex(data):
  #We will rename certain columns and reindex them as per our requirements
  data = data.rename(columns={'Venue': 'venue', 'batter': 'batsmen', 'bowler': 'bowlers','BattingTeam':'batting_team','BowlingTeam':'bowling_team'})
  data = data.reindex(columns=['venue', 'innings', 'batting_team', 'bowling_team','batsmen','bowlers','total_run'])
  return data

data = rename_reindex(data)

In [ ]:
#We can store our final data in the variable x
x = data

In [ ]:
#Checking the final data, after all initial preprocessing stage
data

,venue,innings,batting_team,bowling_team,batsmen,bowlers,total_run
0,Arun Jaitley Stadium,1,Kings XI Punjab,Delhi Daredevils,"KK Nair,KL Rahul,AJ Finch,MA Agarwal","TA Boult,LE Plunkett,Avesh Khan",51
1,Arun Jaitley Stadium,2,Delhi Daredevils,Kings XI Punjab,"G Gambhir,RR Pant,GJ Maxwell,SS Iyer,PP Shaw","BB Sran,AJ Tye,AS Rajpoot",48
2,Wankhede Stadium,1,Sunrisers Hyderabad,Mumbai Indians,"Shakib Al Hasan,KS Williamson,WP Saha,S Dhawan...","JJ Bumrah,HH Pandya,MJ McClenaghan",51
3,Wankhede Stadium,2,Mumbai Indians,Sunrisers Hyderabad,"E Lewis,RG Sharma,Ishan Kishan,SA Yadav,KH Pandya","Mohammad Nabi,Sandeep Sharma,Shakib Al Hasan",22
4,M.Chinnaswamy Stadium,1,Royal Challengers Bangalore,Chennai Super Kings,"Q de Kock,AB de Villiers,V Kohli","Harbhajan Singh,SN Thakur,DL Chahar",52
...,...,...,...,...,...,...,...
597,"Eden Gardens, Kolkata",2,Lucknow Super Giants,Royal Challengers Bangalore,"Q de Kock,KL Rahul,M Vohra,DJ Hooda","Mohammed Siraj,Shahbaz Ahmed,JR Hazlewood",62
598,"Narendra Modi Stadium, Ahmedabad",1,Royal Challengers Bangalore,Rajasthan Royals,"RM Patidar,V Kohli,F du Plessis","TA Boult,M Prasidh Krishna",46
599,"Narendra Modi Stadium, Ahmedabad",2,Rajasthan Royals,Royal Challengers Bangalore,"SV Samson,YBK Jaiswal,JC Buttler","Mohammed Siraj,Shahbaz Ahmed,GJ Maxwell,JR Haz...",67
600,"Narendra Modi Stadium, Ahmedabad",1,Rajasthan Royals,Gujarat Titans,"SV Samson,YBK Jaiswal,JC Buttler","LH Ferguson,Yash Dayal,Mohammed Shami,Rashid Khan",44


Now the data is finally ready as per the requirements. However, there is still some cleaning to do. The IPL has changed over the past few years and we need to adjust our data as per the changes, as much as possible


##f) Stadium Names Cleaning
Same stadiums have been stored in multiple names, hence we have to change those names to a common name for commonality

In [ ]:
def stadium_cleaning(x):
  #We use replace function to change all same stadiums to a common name
  x['venue'] = x['venue'].replace(['Punjab Cricket Association Stadium, Mohali','Punjab Cricket Association IS Bindra Stadium'],'Punjab Cricket Association Stadium')
  x['venue'] = x['venue'].replace(['Feroz Shah Kotla','Arun Jaitley Stadium, Delhi','Arun Jaitley Stadium'],'Arun Jaitley Stadium')
  x['venue'] = x['venue'].replace(['M.Chinnaswamy Stadium','M Chinnaswamy Stadium'],'M.Chinnaswamy Stadium')
  x['venue'] = x['venue'].replace(['Eden Gardens','Eden Gardens, Kolkata'],'Eden Gardens')
  x['venue'] = x['venue'].replace(['Maharashtra Cricket Association Stadium','Maharashtra Cricket Association Stadium, Pune'],'Maharashtra Cricket Association Stadium')
  x['venue'] = x['venue'].replace(['Dr DY Patil Sports Academy','Dr DY Patil Sports Academy, Mumbai'],'Dr DY Patil Sports Academy')
  x['venue'] = x['venue'].replace(['MA Chidambaram Stadium, Chepauk','MA Chidambaram Stadium, Chepauk, Chennai'],'MA Chidambaram Stadium')
  x['venue'] = x['venue'].replace(['Sheikh Zayed Stadium','Zayed Cricket Stadium, Abu Dhabi'],'Zayed Cricket Stadium')
  x['venue'] = x['venue'].replace(['Wankhede Stadium','Wankhede Stadium, Mumbai'],'Wankhede Stadium')
  x['venue'] = x['venue'].replace(['Rajiv Gandhi International Stadium, Uppal','Rajiv Gandhi International Stadium'],'Rajiv Gandhi International Stadium')
  x['venue'] = x['venue'].replace(['Narendra Modi Stadium, Ahmedabad','Sardar Patel Stadium, Motera'],'Narendra Modi Stadium')
  x['venue'] = x['venue'].replace(['Brabourne Stadium, Mumbai'],'Brabourne Stadium')


  return x

x = stadium_cleaning(x)


In [ ]:
x['venue'].unique()

array(['Arun Jaitley Stadium', 'Wankhede Stadium',
       'M.Chinnaswamy Stadium', 'Rajiv Gandhi International Stadium',
       'Maharashtra Cricket Association Stadium',
       'Sawai Mansingh Stadium', 'Eden Gardens', 'Holkar Cricket Stadium',
       'MA Chidambaram Stadium', 'Punjab Cricket Association Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Zayed Cricket Stadium', 'Dubai International Cricket Stadium',
       'Sharjah Cricket Stadium', 'Narendra Modi Stadium',
       'Brabourne Stadium', 'Dr DY Patil Sports Academy'], dtype=object)

## g) Changing Batting/Bowling Team
Since we have seen how team names have changed over the years, we will now implement the same process for teams as well

In [ ]:
def teams_cleaning(x):
  x['batting_team'] = x['batting_team'].replace(['Pune Warriors','Rising Pune Supergiants','Rising Pune Supergiant'],'Rising Pune Supergiants')
  x['bowling_team'] = x['bowling_team'].replace(['Pune Warriors','Rising Pune Supergiants','Rising Pune Supergiant'],'Rising Pune Supergiants')
  x['batting_team'] = x['batting_team'].replace(['Gujarat Lions','Gujarat Titans'],'Gujarat Titans')
  x['bowling_team'] = x['bowling_team'].replace(['Gujarat Lions','Gujarat Titans'],'Gujarat Titans')
  x['batting_team'] = x['batting_team'].replace(['Delhi Daredevils','Delhi Capitals'],'Delhi Capitals')
  x['bowling_team'] = x['bowling_team'].replace(['Delhi Daredevils','Delhi Capitals'],'Delhi Capitals')
  x['batting_team'] = x['batting_team'].replace(['Deccan Chargers','Sunrisers Hyderabad'],'Sunrisers Hyderabad')
  x['bowling_team'] = x['bowling_team'].replace(['Deccan Chargers','Sunrisers Hyderabad'],'Sunrisers Hyderabad')
  x['batting_team'] = x['batting_team'].replace(['Kings XI Punjab','Punjab Kings'],'Punjab Kings')
  x['bowling_team'] = x['bowling_team'].replace(['Kings XI Punjab','Punjab Kings'],'Punjab Kings')

  return x

x = teams_cleaning(x)

## h) Seggregating present teams only
Now, let us use only those entries from our data which have IPL 2023 edition teams.

In [ ]:
def present_teams_wise(x):
  teams = ['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Punjab Kings', 'Rajasthan Royals',
       'Delhi Capitals', 'Mumbai Indians', 'Sunrisers Hyderabad',
       'Gujarat Titans', 'Lucknow Super Giants']
  #We use isin() function to cull out those entries which are required
  x = x[x['batting_team'].isin(teams)]
  x = x[x['bowling_team'].isin(teams)]

  return x

x = present_teams_wise(x)

## i) Transforming Data Using HashLib
We need to convert categorical nominal values in such a way that they reproduce the same results each time. For that, we use hashing function

In [ ]:
def hashing(x):
  import hashlib
  for col in ['venue', 'batting_team', 'bowling_team', 'batsmen','bowlers']:
    hashed_col = x[col].apply(lambda i: hash(i)) #Here, we use lambda to create a numeric hash value for each entry in all categorical values and replace the string value with it's numeric counterpart
    x[col] = hashed_col
  return x

x = hashing(x)

#4) Train-Test Split

## a) Breaking the data between input and output variable
We need to do this to set up the base for supervised learning trainning on various models

In [ ]:
X = x.drop('total_run',axis = 1)
y = x['total_run']


In [ ]:
X.columns

Index(['venue', 'innings', 'batting_team', 'bowling_team', 'batsmen',
       'bowlers'],
      dtype='object')

In [ ]:
input = pd.read_csv('/content/input_test_file.csv')

In [ ]:
input.columns

Index(['venue', 'innings', 'batting_team', 'bowling_team', 'batsmen',
       'bowlers'],
      dtype='object')

In [ ]:
y

0      51
1      48
2      51
3      22
4      52
       ..
597    62
598    46
599    67
600    44
601    31
Name: total_run, Length: 602, dtype: int64

## b) Using this output to create train test sets

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1, shuffle = True)


# 5) Training Models
Using multiple models to train our data and checking their performance using the metric 'mean_absolute_error'

In [ ]:
#Creating dictionary to store model scores 
models = dict()
m = dict()

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

r = Ridge(solver = 'sag')
r.fit(x_train,y_train)
y_pred_r = r.predict(x_test)
y_pred_r_train = r.predict(x_train)

mae_r_train = mean_absolute_error(y_train,y_pred_r_train)
mae_r_test = mean_absolute_error(y_test,y_pred_r)

m['Ridge'] = mae_r_train
models['Ridge'] = mae_r_test
print('MAE on train is:',mae_r_train)
print('MAE on test is:',mae_r_test)


MAE on train is: 9.892778576412669
MAE on test is: 9.885337536301453


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

l = Lasso()
l.fit(x_train,y_train)
y_pred_l = l.predict(x_test)
y_pred_l_train = l.predict(x_train)

mae_l_train = mean_absolute_error(y_train,y_pred_l_train)
mae_l_test = mean_absolute_error(y_test,y_pred_l)

m['Lasso'] = mae_l_train
models['Lasso'] = mae_l_test
print('MAE on train is:',mae_l_train)
print('MAE on test is:',mae_l_test)


MAE on train is: 9.883107018437222
MAE on test is: 9.865668328324844


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.989e+04, tolerance: 8.113e+00
  model = cd_fast.enet_coordinate_descent(


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lr = LinearRegression(fit_intercept = True)
lr.fit(x_train,y_train)
y_pred_lr = lr.predict(x_test)
y_pred_lr_train = lr.predict(x_train)

mae_lr_train = mean_absolute_error(y_train,y_pred_lr_train)
mae_lr_test = mean_absolute_error(y_test,y_pred_lr)

m['LinearRegression'] = mae_lr_train
models['LinearRegression'] = mae_lr_test
print('MAE on train is:',mae_lr_train)
print('MAE on test is:',mae_lr_test)



MAE on train is: 9.892778234267634
MAE on test is: 9.885355100820522


In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error

sgd = SGDRegressor(random_state = 1)
sgd.fit(x_train,y_train)
y_pred_sgd = sgd.predict(x_test)
y_pred_sgd_train = sgd.predict(x_train)

mae_sgd_train = mean_absolute_error(y_train,y_pred_sgd_train)
mae_sgd_test = mean_absolute_error(y_test,y_pred_sgd)

#models['SGDRegressor'] = mae_sgd_test
print('MAE on train is:',mae_sgd_train)
print('MAE on test is:',mae_sgd_test)


MAE on train is: 4.002674362709321e+46
MAE on test is: 3.8314723385703554e+46


In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_error

byr = BayesianRidge(fit_intercept = True, verbose = 3)
byr.fit(x_train,y_train)
y_pred_byr = byr.predict(x_test)
y_pred_byr_train = byr.predict(x_train)

mae_byr_train = mean_absolute_error(y_train,y_pred_byr_train)
mae_byr_test = mean_absolute_error(y_test,y_pred_byr)

m['BayesianRidge'] = mae_byr_train
models['BayesianRidge'] = mae_byr_test
print('MAE on train is:',mae_byr_train)
print('MAE on test is:',mae_byr_test)


Convergence after  2  iterations
MAE on train is: 9.893718554499674
MAE on test is: 9.910253657573927


In [ ]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error

kr = KernelRidge()
kr.fit(x_train,y_train)
y_pred_kr = kr.predict(x_test)
y_pred_kr_train = kr.predict(x_train)

mae_kr_train = mean_absolute_error(y_train,y_pred_kr_train)
mae_kr_test = mean_absolute_error(y_test,y_pred_kr)

#models['KernelRidge'] = mae_kr_test
print('MAE on train is:',mae_kr_train)
print('MAE on test is:',mae_kr_test)

MAE on train is: 37.245572510783525
MAE on test is: 37.989009711166375


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

hgb = HistGradientBoostingRegressor(random_state = 1)
hgb.fit(x_train,y_train)
y_pred_hgb = hgb.predict(x_test)
y_pred_hgb_train = hgb.predict(x_train)

mae_hgb_train = mean_absolute_error(y_train,y_pred_hgb_train)
mae_hgb_test = mean_absolute_error(y_test,y_pred_hgb)

m['HistGradientBoostingRegressore'] = mae_hgb_train
models['HistGradientBoostingRegressore'] = mae_hgb_test
print('MAE on train is:',mae_hgb_train)
print('MAE on test is:',mae_hgb_test)

MAE on train is: 5.051521827558592
MAE on test is: 10.398374203698957


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

mlp = MLPRegressor(random_state = 1)
mlp.fit(x_train,y_train)
y_pred_mlp = mlp.predict(x_test)
y_pred_mlp_train = mlp.predict(x_train)

mae_mlp_train = mean_absolute_error(y_train,y_pred_mlp_train)
mae_mlp_test = mean_absolute_error(y_test,y_pred_mlp)

#models['MLPRegressor'] = mae_mlp_test
print('MAE on train is:',mae_mlp_train)
print('MAE on test is:',mae_mlp_test)

MAE on train is: 3.622219157462051e+16
MAE on test is: 5.408934648472507e+16


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error

et = ExtraTreesRegressor(random_state = 1)
et.fit(x_train,y_train)
y_pred_et = et.predict(x_test)
y_pred_et_train = et.predict(x_train)

mae_et_train = mean_absolute_error(y_train,y_pred_et_train)
mae_et_test = mean_absolute_error(y_test,y_pred_et)

m['ExtraTreesRegressor'] = mae_et_train
models['ExtraTreesRegressor'] = mae_et_test
print('MAE on train is:',mae_et_train)
print('MAE on test is:',mae_et_test)

MAE on train is: 0.0026611226611226637
MAE on test is: 9.285702479338843


In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_absolute_error

br = BaggingRegressor(random_state = 1,verbose = 1)
br.fit(x_train,y_train)
y_pred_br = br.predict(x_test)
y_pred_br_train = br.predict(x_train)

mae_br_train = mean_absolute_error(y_train,y_pred_br_train)
mae_br_test = mean_absolute_error(y_test,y_pred_br)

m['BaggingRegressor'] = mae_br_train
models['BaggingRegressor'] = mae_br_test
print('MAE on train is:',mae_br_train)
print('MAE on test is:',mae_br_test)

MAE on train is: 3.530977130977131
MAE on test is: 9.608264462809917


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

rfr = RandomForestRegressor(random_state = 1,n_estimators = 100,criterion = 'absolute_error')
rfr.fit(x_train,y_train)
y_pred_rfr = rfr.predict(x_test)
y_pred_rfr_train = rfr.predict(x_train)

mae_rfr_train = mean_absolute_error(y_train,y_pred_rfr_train)
mae_rfr_test = mean_absolute_error(y_test,y_pred_rfr)

m['RandomForestRegressor'] = mae_rfr_train
models['RandomForestRegressor'] = mae_rfr_test
print('MAE on train is:',mae_rfr_train)
print('MAE on test is:',mae_rfr_test)

MAE on train is: 3.5666632016632014
MAE on test is: 8.769090909090911


In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error

ab = AdaBoostRegressor(random_state = 1,loss = 'linear',n_estimators = 400, learning_rate = 0.04)
ab.fit(x_train,y_train)
y_pred_ab = ab.predict(x_test)
y_pred_ab_train = ab.predict(x_train)

mae_abr_train = mean_absolute_error(y_train,y_pred_ab_train)
mae_abr_test = mean_absolute_error(y_test,y_pred_ab)

m['AdaBoostRegressor'] = mae_abr_train
models['AdaBoostRegressor'] = mae_abr_test
print('MAE on train is:',mae_abr_train)
print('MAE on test is:',mae_abr_test)

MAE on train is: 8.802057097744495
MAE on test is: 8.42507474256028


In [ ]:

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

gb = GradientBoostingRegressor(random_state = 1,n_estimators = 400, max_depth = 3,loss = 'absolute_error',learning_rate = 0.04)
gb.fit(x_train,y_train)
y_pred_gb = gb.predict(x_test)
y_pred_gb_train = gb.predict(x_train)

mae_gbr_train = mean_absolute_error(y_train,y_pred_gb_train)
mae_gbr_test = mean_absolute_error(y_test,y_pred_gb)

m['GradientBoostingRegressor'] = mae_gbr_train
models['GradientBoostingRegressor'] = mae_gbr_test
print('MAE on train is:',mae_gbr_train)
print('MAE on test is:',mae_gbr_test)



MAE on train is: 7.242375014295799
MAE on test is: 8.728156297104244


In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_absolute_error

xgb = XGBRegressor(random_state = 1)
xgb.fit(x_train,y_train)
y_pred_xgb = xgb.predict(x_test)
y_pred_xgb_train = xgb.predict(x_train)

mae_xgb_train = mean_absolute_error(y_train,y_pred_xgb_train)
mae_xgb_test = mean_absolute_error(y_test,y_pred_xgb)

m['XGBRegressor'] = mae_xgb_train
models['XGBRegressor'] = mae_xgb_test
print('MAE on train is:',mae_xgb_train)
print('MAE on test is:',mae_xgb_test)


MAE on train is: 0.3882173292354338
MAE on test is: 9.966916516792676


In [ ]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

lgb = XGBRegressor(random_state = 1)
lgb.fit(x_train,y_train)
y_pred_lgb = lgb.predict(x_test)
y_pred_lgb_train = lgb.predict(x_train)

mae_lgb_train = mean_absolute_error(y_train,y_pred_lgb_train)
mae_lgb_test = mean_absolute_error(y_test,y_pred_lgb)

m['LGBMRegressor'] = mae_lgb_train
models['LGBMRegressor'] = mae_lgb_test
print('MAE on train is:',mae_lgb_train)
print('MAE on test is:',mae_lgb_test)


MAE on train is: 0.3882173292354338
MAE on test is: 9.966916516792676


## a) Trying combinations of models

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_absolute_error

vr = VotingRegressor([('gb',gb),('ab',ab)])
vr.fit(x_train,y_train)
y_pred_vr = vr.predict(x_test)
y_pred_vr_train = vr.predict(x_train)

mae_vr_train = mean_absolute_error(y_train,y_pred_vr_train)
mae_vr_test = mean_absolute_error(y_test,y_pred_vr)

m['VotingRegressor'] = mae_vr_train
models['VotingRegressor'] = mae_vr_test
print('MAE on train is:',mae_vr_train)
print('MAE on test is:',mae_vr_test)

MAE on train is: 7.989958576009354
MAE on test is: 8.536364670121943


In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_error

ab = AdaBoostRegressor(random_state = 1,loss = 'linear',n_estimators = 400, learning_rate = 0.04)
gb = GradientBoostingRegressor(random_state = 1,n_estimators = 400, max_depth = 3,loss = 'absolute_error',learning_rate = 0.04)
rfr = RandomForestRegressor(random_state = 1,n_estimators = 100,criterion = 'absolute_error')

byr = BayesianRidge()

sr = StackingRegressor([('gb',gb),('ab',ab),('rfr',rfr)],final_estimator = byr)
sr.fit(x_train,y_train)
y_pred_sr = sr.predict(x_test)
y_pred_sr_train = sr.predict(x_train)

mae_sr_train = mean_absolute_error(y_train,y_pred_sr_train)
mae_sr_test = mean_absolute_error(y_test,y_pred_sr)

m['StackingRegressor'] = mae_sr_train
models['StackingRegressor'] = mae_sr_test
print('MAE on train is:',mae_sr_train)
print('MAE on test is:',mae_sr_test)

MAE on train is: 5.604497195049007
MAE on test is: 8.579578436102235


## b) Best Model Selection

In [ ]:
diff = dict()
for i in m.keys():
  diff[i] = abs(m[i]-models[i])
min(diff, key=diff.get)

'LinearRegression'

This shows that the difference between trainning and test error is the lowest. Now let us look at the test error

In [ ]:
models['VotingRegressor']

8.536364670121943

If we look at Stacking Regressor, it has a similar value.

In [ ]:
models['StackingRegressor']

8.579578436102235

### i) Lowest Trainning Error

In [ ]:
min(m, key=m.get)

'ExtraTreesRegressor'

So, trainning error for ExtraTreesRegressor is the lowest. Let us see the difference between the training and test error

In [ ]:
m['ExtraTreesRegressor']

0.0026611226611226637

In [ ]:
models['ExtraTreesRegressor']

9.285702479338843

As we can see, the difference is quite high, as shown below.

In [ ]:
diff['ExtraTreesRegressor']

9.283041356677721

Thus, the model is overfitting heavily.

### ii) Lowest Test Error

In [ ]:
min(models, key=models.get)

'AdaBoostRegressor'

This shows that the test error is lowest for Bagging Regressor. However, let us see the difference between the training and test error

In [ ]:
models['BaggingRegressor']

9.608264462809917

In [ ]:
m['BaggingRegressor']

3.530977130977131

In [ ]:
diff['BaggingRegressor']

6.077287331832785

As we can see, the difference is quite high. Hence, this is not the best suitable model. A trainning error is low but test is almost twice of trainning error in Bagging Regressor. 
Thus, this model is overfitting.

### iii) Final Model
Hence, we will use **stacking regresor** to make predictions. 

1. We can also use the voting regressor as well. However, since 
stacking has a lower MAE in train data set and the error on test dataset is marginally high than voting regressor, we will go for that.

2. Kindly note that either of the two models i.e. Voting and Stacking Regressor could be the ideal model.

In [ ]:
#Creating the final model
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_error

ab = AdaBoostRegressor(random_state = 1,loss = 'linear',n_estimators = 400, learning_rate = 0.04)
gb = GradientBoostingRegressor(random_state = 1,n_estimators = 400, max_depth = 3,loss = 'absolute_error',learning_rate = 0.04)
rfr = RandomForestRegressor(random_state = 1,n_estimators = 100,criterion = 'absolute_error')
byr = BayesianRidge()

sr_final = StackingRegressor([('gb',gb),('ab',ab),('rfr',rfr)],final_estimator = byr)
sr_final.fit(X,y)

StackingRegressor(estimators=[('gb',
                               GradientBoostingRegressor(learning_rate=0.04,
                                                         loss='absolute_error',
                                                         n_estimators=400,
                                                         random_state=1)),
                              ('ab',
                               AdaBoostRegressor(learning_rate=0.04,
                                                 n_estimators=400,
                                                 random_state=1)),
                              ('rfr',
                               RandomForestRegressor(criterion='absolute_error',
                                                     random_state=1))],
                  final_estimator=BayesianRidge())

## Trial 1: SRH vs CSK 2023

In [ ]:
ip = pd.read_csv('/content/ipl_latest - Sheet1.csv')
ip = hashing(ip)
y_p_00 = sr_final.predict(ip)
y_p_00_int =  [round(y) for y in y_p_00] #Prediction in Integer, since runs scored can only be whole numbers

actual_ip = [45,60] #Actual Score 
error_ip = mean_absolute_error(actual_ip,y_p_00_int) #Calculating Mean Absolute Error
error_ip

7.5

## Trial2: RR vs GT 2022 (Finals)

In [ ]:
ip2 = pd.read_csv('/content/ipl_22_final - Sheet1.csv')
ip2 = hashing(ip)
y_p_0 = sr_final.predict(ip2)
y_p_0_int =  [round(y) for y in y_p_0] #Prediction in Integer, since runs scored can only be whole numbers

actual_ip2 = [44,31] #Actual Score
error_ip2 = mean_absolute_error(actual_ip2,y_p_0_int) #Calculating Mean Absolute Error
error_ip2


8.5